In [5]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,BatchNormalization,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [8]:
data = pd.read_csv(r"C:\Users\Dell\Desktop\zzzzz\projet tutoré\interpetation using python\sentiment1\Sentiment.csv",encoding = "ISO-8859-1")
# Keeping only the neccessary columns
data = data[['text','sentiment']]

ParserError: Error tokenizing data. C error: Expected 21 fields in line 24, saw 29


In [7]:
data.info()

NameError: name 'data' is not defined

In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.add(Dense(2,activation='softmax'))
model.add(BatchNormalization())  
model.add(Dense(4, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 2)

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
#twt=pd.read_csv("")
twt=['I m happy']
tokenizer = Tokenizer(num_words=2000, split=' ')
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

In [ ]:
Tokenizer.texts_to_sequences?

In [ ]:
path_to_model=r"C:\Users\Dell\Desktop\zzzzz\projet tutoré\interpetation using python\SentiModel\model.pb"
model.save(path_to_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("SentiModel.tflite", "wb").write(tflite_model)

In [ ]:
model = keras.models.load_model(path_to_model)

In [ ]:
import os

In [ ]:
data_dir=r"C:\Users\Dell\Desktop\zzzzz\projet tutoré\interpetation using python\SentiModel"
def load_data():
    text =""
    for r,d,f in os.walk(data_dir):
        file=os.path.join(r,f[0]);
    text= open(file, "r")
    return text.read()

SentiForPred = load_data()

In [ ]:
twt = tokenizer.texts_to_sequences(SentiForPred)
twt=['i m happy']
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")